# Podstawy podejmowania decyzji projekt
283018

## Opis

1. Plansza (macierz) nxn zapełniona symbolami.
2. Każda komórka macierzy zawiera jeden symbol w formacie heksadecymalnym (nie ma to zasadniczego znaczenia i można je zamienić na liczby całkowite lub symbole ascii) z pewnej ograniczonej puli.
3. Symbole w różnych komórkach macierzy mogą się powtarzać.
4. Jest lista "demonów" - sekwencji różnej długości złożonych z symboli z tej samej puli (przy czym jedna sekwencja może zawierać inną sekwencję w części albo w pełni).
5. Jest ograniczony "buffor" - maksymalna ilość symboli które można aktywować/wybrać na macierzy.
6. Wybierać symbole z planszy można według określonych zasad:<br>
	a) w pierwszym kroku można wybrać dowolny symbol z pierwszego *wiersza*, na przykład (0, 4)<br>
	b) w drugim kroku symbol z tej samej *kolumny* co poprzednio wybrany symbol, np. (3, 4)<br>
	c) w trzecim znowu z tego samego *wiersza* co poprzednio wybrany, np. (5, 4)<br>
	Na ogół, jeśli krok jest nieparzysty to wybór się odbywa w odpowiednim (zgodnym z poprzednio wybranym) *wierszu*, jeśli parzysty to w *kolumnie*.
7. Można wybierać symbol z tej samej komórki macierzy tylko jednokrotnie
8. Jeśli zostały kolejno wybrane symbole odpowiadające "demonowi" to ta sekwencja jest "aktywowana".
9. Można wybierać symbole niebędące elementem żadnego "demona".
10. Macierz jest generowana w taki sposób by każdy pojedynczy "demon" był możliwy do aktywacji
<br>*[dodatkowe zasady]*
11. Sekwencja demona może rozpoczynać się w dowolnym miejscu sekwencji buffera, o ile jest zawarta w pełni to demon jest aktywowany
<br> - np. demon: [1, 2], buffer [3, 4, 1, 2], demon jest aktywowany
12. Każdy demon może być aktywowany tylko raz
<br> - np. demon [1, 2], buffer: [1, 2, 1, 2], demon będzie aktywowany ale punkty za jego aktywację będą zdobyte tylko jednokrotnie
13. Jeśli sekwencje demonów się zawierają te demony mogą być aktywowane tą samą sekwencją buffera, jeśli każda z nich jest w pełni zawarta
<br> - np. demony: [1, 2], [3, 1, 2], sekwencja buffera: [3, 1, 2], oba demony bedą aktywowane (za oba będą zdobyte punkty)

# Instalacja

In [ ]:
!pip install gurobipy
# !pip install icecream


In [ ]:
import gurobipy as gp
# import numpy as np  # noqa: F401
# from icecream import ic  # noqa: F401
from pprint import pprint as pp #noqa: F401

# ------

## Przykłądowe dane wejściowe (tylko jeden zestaw do wybrania)

In [ ]:
buffer_size = 8
demons = [
    [1, 3, 4, 1],
    [3, 1, 2],
    [1, 3, 1, 2, 4],
]
d_costs = [3, 2, 5]

matrix = [
    [1, 3, 3],
    [4, 2, 1],
    [3, 1, 4],
]

In [ ]:
buffer_size = 1
demons = [
    [1],
]
d_costs = [5]
matrix = [
    [1],
]

In [ ]:
buffer_size = 5
demons = [
    [2, 2, 2, 2, 2],
    [1, 3],
]
d_costs = [10, 3]
matrix = [
    [1, 2, 3],
    [2, 2, 1],
    [2, 3, 2],
]

In [ ]:
buffer_size = 6
demons = [
    [1, 2],
    [2, 1]
]
d_costs = [2, 3]
matrix = [
    [1, 2],
    [2, 1],
]

In [ ]:
buffer_size = 8
demons = [
    [1, 3],
    [4, 2]
]
d_costs = [5, 5]
matrix = [
    [1, 3],
    [4, 2]
]

In [ ]:
buffer_size = 7
demons = [
    [1, 4, 3, 2, 1],
    [2, 2, 2],
]
d_costs = [15, 5]
matrix = [
    [1, 2, 3, 4],
    [4, 3, 2, 1],
    [1, 2, 3, 4],
    [4, 3, 2, 1]
]

In [ ]:
# przykład prosto z gry
buffer_size = 8
demons = [
    [1, 2],
    [3, 4],
	[5, 1, 2]
]
d_costs = [1, 2, 3]
matrix = [
    [3, 1, 5, 5, 3, 5, 2],
    [5, 6, 2, 2, 5, 5, 1],
    [5, 5, 4, 2, 6, 5, 2],
    [1, 2, 1, 3, 2, 2, 1],
    [1, 5, 2, 4, 6, 6, 4],
    [3, 1, 3, 5, 5, 2, 3],
    [3, 5, 1, 5, 6, 3, 2],
]

In [ ]:
# jeszcze jeden przykład z gry (trudniejszy)
buffer_size = 12
demons = [
    [9, 11, 6, 10],
    [1, 6, 8, 1],
    [9, 8, 4, 6],
    [10, 7, 4, 4, 11],
    [6, 8, 7, 1],
    [1, 4, 10, 3, 11],
    [3, 2, 4, 10, 1],
    [9, 8, 1, 9, 10, 3],
    [10, 9, 8, 10, 2, 6],
    [1, 10, 11, 1, 1, 3, 11],
    [4, 2, 7, 11, 10, 4],
]
d_costs = [4, 3, 3, 4, 4, 5, 5, 7, 8, 10, 5]
matrix = [
    [10, 6, 8, 7, 7, 2, 1, 8],
    [2, 4, 1, 9, 4, 6, 11, 6],
    [6, 1, 4, 7, 1, 7, 10, 9],
    [3, 4, 1, 4, 10, 4, 3, 2],
    [8, 9, 10, 11, 1, 1, 8, 4],
    [11, 10, 1, 11, 6, 1, 11, 1],
    [9, 1, 8, 11, 1, 6, 3, 1],
    [2, 2, 9, 3, 1, 10, 9, 6]
]

## Definicja modelu i wyniki

In [ ]:
# aktualna konfiguracja (stałe)
d_amo = len(demons)
d_lengths = [len(i) for i in demons]
matrix_size = len(matrix)
max_points = sum(d_costs)
if not all(len(row) == matrix_size for row in matrix):
    raise ValueError("Matrix must have same dimensions")

# wynagrodzenie za niezużycie buffera
# unused_reward = 0.1
unused_reward = 0.1 * (max_points / d_amo)



model = gp.Model("Breach Protocol")
model.setParam("OutputFlag", 0)



# wybór drogi
x = model.addVars(matrix_size, matrix_size, buffer_size, vtype=gp.GRB.BINARY, name='x')

# co najwyżej jedna komórka wybierana w każdym kroku
for t in range(buffer_size):
    model.addConstr(x.sum('*', '*', t) <= 1, name=f'one_cell_per_step_{t}')

# ciągłość drogi - nie można wybrać komórki po niewybranej komórki (ale można nie wybierać na końcu)
for t in range(1, buffer_size):
    prev = x.sum('*', '*', t-1)
    curr = x.sum('*', '*', t)
    model.addConstr(curr <= prev, name=f'continuous_path_at_{t}')

# komórka może być wybrana co najwyżej raz
for i in range(matrix_size):
    for j in range(matrix_size):
        model.addConstr(x.sum(i, j, '*') <= 1, name=f'cell_{i}_{j}_once')

# maksymalna ilość kroków
used_buffer = x.sum('*', '*', '*')
model.addConstr((used_buffer <= buffer_size), name='max_steps')

# rozpoczęcie zawsze w pierwszym rzędzie - dla wszystkich rzędów poza pierwszym w kroku 0 nie można wybierać komórki
model.addConstrs((x.sum(i, '*', 0) == 0 for i in range(1, matrix_size)), name='start_in_first_row')

# poruszanie się po macierzy zgodnie z zasadami rząd/kolumna
# TODO: chyba da się zmniejszyć ilość potrzebnych ograniczeń
# TODO: może użyć Var prev_row i prev_col z addGenConstrIndicator ?
for t in range(1, buffer_size):
    for i in range(matrix_size):
        for j in range(matrix_size):
            if t % 2 == 1:
                # dla dowolnej komórki w kroku t musi być wybrana dowolna komórka w tej samej *kolumnie* w kroku t-1
                model.addConstr(x[i, j, t] <= x.sum('*', j, t-1), name=f'move_rule_col_{i}_{j}_step_{t}')
            else:
                model.addConstr(x[i, j, t] <= x.sum(i, '*', t-1), name=f'move_rule_row_{i}_{j}_step_{t}')


# bezpośrednio sekwencja buffera
# TODO: trzeba albo wykluczyć 0 z symboli albo dodać default na -1
buffer_seq = [
    gp.quicksum(
        matrix[i][j] * x[i, j, t]
        for i in range(matrix_size)
        for j in range(matrix_size)
        )
    for t in range(buffer_size)
]


# lista, dla każdego demona jest słownik pozycji w sekwencji buffera gdzie ten demon może się zaczynać : is_valid
z = []
for i in range(d_amo):
    curr_len = d_lengths[i]
    valid_p = buffer_size - curr_len + 1
    z_i = {}
    for p in range(valid_p):
        z_i[p] = model.addVar(vtype=gp.GRB.BINARY, name=f'z_{i}_{p}')
    z.append(z_i)
# [{pos: is_valid}, ...] ∀ demon (according to demon length)


for i in range(d_amo):                              # ∀ demona
    curr_len = d_lengths[i]
    for p in range(buffer_size - curr_len + 1):     # ∀ valid starting pos
        for s in range(curr_len):                   # ∀ symbolu demona
            t = p + s       # krok (pozycja w buffer)
            model.addGenConstrIndicator(
                z[i][p],                        # jeśli dla tego demona dla tej pozycji startowej is_valid
                1,                              # == 1 (True)
                buffer_seq[t] == demons[i][s],  # to w buffer (na odpowiedniej pozycji) MUSI być symbol demona
                name=f'indicator(demon_{i}_pos_{p}_symbol_{s})'
            )


# wybrane demony
y = model.addVars(d_amo, vtype=gp.GRB.BINARY, name='y')


# TODO: czy optymalizator nie będzie chciał aktywować tego samego demona kilkukrotnie? (chociaż i tak punkty w y można zdobyć tylko jednokrotnie, idk)
# jeśli chociaż jedna z starting_pos valid to y[i] == 1
for i in range(d_amo):
    curr_len = d_lengths[i]
    valid_p = buffer_size - curr_len + 1

    # sekwencja demona może występować w buffer kilkakrotnie np. demon=[1, 3], buffer = [1, 3, 1, 3]
    # y[i] == 0 jeśli *żadna* z z[i][p] nie == 1
    model.addConstr(y[i] <= gp.quicksum(z[i][p] for p in range(valid_p)), name=f'active_y_{i}_at_least_one')
    # jeśli dowolna z z[i][p] == 1, y[i] też == 1
    for p in range(valid_p):
        model.addConstr(y[i] >= z[i][p], name=f'active_y_{i}_for_{p}')



# maksymalizacja punktów (obv) + wynagrodzenie za niezużycie buffera
model.setObjective(gp.quicksum(d_costs[i] * y[i] for i in range(d_amo)) + unused_reward*(buffer_size - used_buffer), gp.GRB.MAXIMIZE)
model.optimize()



'''
Wypisywanie wyników
'''
if model.status != gp.GRB.OPTIMAL:
    raise ValueError(f"Optimization failed, model.status={model.status}")

used_buffer_num = int(used_buffer.getValue())

buffer_nums = [int(buffer_seq[t].getValue()) for t in range(buffer_size)]
print(f"\nSekwencja buffera ({used_buffer_num}/{buffer_size}):")
pp(buffer_nums)

z_nums = []
for i in range(len(z)):
    z_nums.append({})
    for j, k in z[i].items():
        z_nums[i][j] = True if k.x > 0.5 else False
# pp(z_nums)

x_steps_nums = [[None for _ in range(matrix_size)] for _ in range(matrix_size)]
for i in range(matrix_size):
    for j in range(matrix_size):
        for t in range(buffer_size):
            if x[i, j, t].x > 0.5:
                x_steps_nums[i][j] = t
print("\nKolejno kroki na macierzy:")
for i in x_steps_nums:
    pp(i)

x_path_cells_nums = []
for t in range(buffer_size):
    for i in range(matrix_size):
        for j in range(matrix_size):
            if x[i, j, t].x > 0.5:
                x_path_cells_nums.append((i, j))
print("\nKoordynaty kolejno wybranych kroków:")
pp(x_path_cells_nums)

x_cell_chosen_num = [[0 for _ in range(matrix_size)] for _ in range(matrix_size)]
for i in range(matrix_size):
    for j in range(matrix_size):
        for t in range(buffer_size):
            if x[i, j, t].x > 0.5:
                x_cell_chosen_num[i][j] += 1
# pp(x_cell_chosen_num)

y_nums = []
for i in y.values():
    y_nums.append(bool(int(i.x)))
print("\nKolejno czy demon został aktywowany:")
pp(y_nums)

y_points_total = sum(d_costs[i] * y_nums[i] for i in range(d_amo))
print("\nZdobyte punkty:")
print(f"{y_points_total}/{max_points}")


